In [2]:
import numpy as np

## 0. Neural Network Setup

This is a very simple step by step demonstration of Layer-wise relevance propagation (LRP) (Montavon, G., Binder, A., Lapuschkin, S., Samek, W., & Müller, K. R. (2019). Layer-wise relevance propagation: an overview. Explainable AI: interpreting, explaining and visualizing deep learning, 193-209).

We start by defining a very simple Neural Network with 3 layers for a binary classification task (Is it class 1 or is it class 2?).

1. Input Layer: Layer of the network with 3 neurons
2. Hidden Layer: Layer with 3 neurons, ReLU activation function
3. Output Layer: Layer with 1 neuron, Sigmoid Activation Function, returns probability p for class 1

In [3]:
w_hidden = np.asarray([[0.3, 0.2, 0.4],
            [0.6, 0.1, 0.5],
            [0.8, 0.6, 0.2]])

w_output = np.asarray([[0.3],
            [0.1],
            [0.4]])

b_hidden = np.asarray([0.5])

b_output = np.asarray([0.1])

input = np.asarray([[1.0], [0.8], [0.2]])

def ReLU(Z):
    A = np.maximum(0, Z)
    return A

def Sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A


![Initial State](step_by_step_img/initial.png)

## 1. Forward Pass to calculate Relevance Score

First step is a forward pass to calculate the Relevance score, which equals the activation of the output layer. Forward pass in neural networks is a dot-product of input values and weights plus a bias. We will perform two passes, one from Input to Hidden layer and one from Hidden to Output layer.

In [4]:
input_hidden = np.dot(w_hidden, input) + b_hidden
print(input_hidden)

[[1.04]
 [1.28]
 [1.82]]


![Initial State](step_by_step_img/first_pass.png)


After performing the dot product, an activation function is applied. In this case the Rectified Linear Unit (see bottom of Hidden layer)

In [5]:
output_hidden = ReLU(input_hidden)
print(output_hidden)

[[1.04]
 [1.28]
 [1.82]]


![Initial State](step_by_step_img/first_activation.png)

In [6]:
input_output = np.dot(w_output.T, output_hidden) + b_output
print(input_output)

[[1.268]]


![Initial State](step_by_step_img/second_pass.png)

In [7]:
output_output = Sigmoid(input_output)
print(np.round(output_output, 2))

[[0.78]]


The sigmoid activation returns a result of 0.78 for the first forward pass, which indicates a 78% probability for class 1.

![Initial State](step_by_step_img/forwardpass_output.png)

## 2. Calculating Relevance scores

After the forward pass, we calculate the relevance scores of each neuron layer-for-layer with the formula given in the paper (see below). The sum over the relevance should be equal for each layer.


In [8]:
relevance = output_output
activation_previous = output_hidden
activation_sum = np.sum(activation_previous)


In [9]:
hidden_relevance = np.round(activation_previous * relevance / activation_sum,2)

In [10]:
print(hidden_relevance)

[[0.2 ]
 [0.24]
 [0.34]]


![Initial State](step_by_step_img/first_relevance_propagation.png)

In [11]:
relevance = hidden_relevance
activation_previous = input
activation_sum = np.sum(input)

In [12]:
input_relevance = []
for i in range(len(activation_previous)):
    input_relevance.append(sum(np.round(relevance * activation_previous[i]  / activation_sum,2)))

input_relevance = np.asarray(input_relevance)

In [13]:
print(input_relevance)

[[0.39]
 [0.32]
 [0.07]]


![Initial State](step_by_step_img/second_relevance_propagation.png)

The final relevance scores of the input neurons are: 1. 0.39, 2. 0.32, and 3. 0.07. The scores show a strong relevance of the first and second input on the classification result.